![head.png](https://github.com/cafawo/FinancialDataAnalytics/blob/master/figures/head.jpg?raw=1)

# Financial Data Analytics in Python

**Prof. Dr. Fabian Woebbeking**</br>
Assistant Professor of Financial Economics

IWH - Leibniz Institute for Economic Research</br>
MLU - Martin Luther University Halle-Wittenberg

fabian.woebbeking@iwh-halle.de

# Homework: Django (web-framework)

You will need a Git/GitHub repository to submit your course deliverables. Consult [**slides.ipynb**](https://github.com/cafawo/FinancialDataAnalytics) for help with the tasks below! If you need further assistance, do not hesitate to open a Q&A at https://github.com/cafawo/FinancialDataAnalytics/discussions

In [2]:
# Packages used in this notebook
import django
import requests

### Task:

Use Deribit's `"/public/get_tradingview_chart_data"` API endpoint to download historical price data for **"BTC-PERPETUAL"** with a resolution of $60$ minutes, reaching as far back as possible.

Hint: https://docs.deribit.com/#public-get_tradingview_chart_data

In [3]:
import requests
import pandas as pd
import time

def fetch_deribit_data(resolution=60):
    url = "https://www.deribit.com/api/v2/public/get_tradingview_chart_data"
    params = {
        'instrument_name': 'BTC-PERPETUAL',
        'resolution': str(resolution),
        'start_timestamp': 0,  # Start from the earliest possible timestamp
        'end_timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }

    response = requests.get(url, params=params)
    data = response.json()

    if 'result' in data:
        return data['result']
    else:
        print("Error fetching data:", data)
        return None

def process_and_save_data(data, filename='btc_perpetual_data.csv'):
    if data:
        df = pd.DataFrame(data)
        df['timestamp'] = pd.to_datetime(df['ticks'], unit='ms')
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")
    else:
        print("No data to save.")

if __name__ == "__main__":
    data = fetch_deribit_data()
    process_and_save_data(data)


Data saved to btc_perpetual_data.csv


### Task:

Create a new Django project (or clone https://github.com/cafawo/StudySnips).

Ad an "api/deribit/" endpoint to your project. This means that you need to modify **view.py** by adding a function to download the data and `return JsonResponse(data, safe=False)`. Also add the endpoint to **urls.py**.

Hint: You can serve data directly through view.py, i.e. you do not need any data base model (models.py) to solve this task.

In [7]:
# myapp/views.py
from django.http import JsonResponse
import requests
import time

def fetch_deribit_data():
    url = "https://www.deribit.com/api/v2/public/get_tradingview_chart_data"
    params = {
        'instrument_name': 'BTC-PERPETUAL',
        'resolution': '60',
        'start_timestamp': 0,
        'end_timestamp': int(time.time() * 1000)
    }

    response = requests.get(url, params=params)
    data = response.json()

    if 'result' in data:
        return data['result']
    else:
        return {"error": "Error fetching data"}

def deribit_data(request):
    data = fetch_deribit_data()
    return JsonResponse(data, safe=False)

# myapp/urls.py
from django.urls import path
from . import views

urlpatterns = [
    path('api/deribit/', views.deribit_data, name='deribit_data'),
]

# myproject/urls.py
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('myapp.urls')),
]



ImportError: attempted relative import with no known parent package

### Task:

Add a Chart.js object to display your Deribit data. You could start by modifying https://github.com/cafawo/StudySnips.git. You will need a template (chart.html) as well as modifications to view.py and urls.py - have a look at the slides.